# 詞性標注練習

In [ ]:
# 安裝
# pip install spacy
# python -m spacy download en_core_web_md

In [1]:
import spacy

In [3]:
# 載入英文模型
nlp = spacy.load("en_core_web_md")

# 要分析的句子
sentence = "List suppliers who supply red parts?"

# 使用 spaCy 處理句子
doc = nlp(sentence)

# 輸出每個詞彙的文本、通用詞性和詳細詞性
for token in doc:
    print(f"{token.text:12} {token.pos_:10} {token.dep_}")

List
suppliers
who
supply
red
parts
?


In [5]:
nlp = spacy.load("en_core_web_md")
sen = "List the students who take the course taught by Frank"
doc = nlp(sen)
for token in doc:
    print(f"{token.text:12} {token.pos_:10} {token.dep_}")

List         VERB       ROOT
the          DET        det
students     NOUN       dobj
who          PRON       nsubj
take         VERB       relcl
the          DET        det
course       NOUN       dobj
taught       VERB       acl
by           ADP        agent
Frank        PROPN      pobj
